<a href="https://colab.research.google.com/github/LenaGeller/retail_demand_analysis/blob/main/data_prep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import requests
import io

# Build the direct download URL from a file ID
def make_drive_url(file_id):
    return f"https://drive.google.com/uc?id={file_id}"

# Helper function to load a CSV from a direct URL
def load_csv_from_url(url):
    response = requests.get(url)
    response.raise_for_status()  # Raises an error if the request fails
    return pd.read_csv(io.StringIO(response.text))

# Dictionary of file IDs for clarity
file_ids = {
    "holiday_events": "1RMjSuqHXHTwAw_PGD5XVjhA3agaAGHDH",
    "items": "1ogMRixVhNY6XOJtIRtkRllyOyzw1nqya",
    "oil": "1Q59vk2v4WQ-Rpc9t2nqHcsZM3QWGFje_",
    "stores": "1Ei0MUXmNhmOcmrlPad8oklnFEDM95cDi",
    "train": "1oEX8NEJPY7wPmSJ0n7lO1JUFYyZjFBRv",
    "transactions": "1PW5LnAEAiL43fI5CRDn_h6pgDG5rtBW_"
}

# Load each CSV using the helper functions
df_holiday_events = load_csv_from_url(make_drive_url(file_ids["holiday_events"]))
df_items          = load_csv_from_url(make_drive_url(file_ids["items"]))
df_oil            = load_csv_from_url(make_drive_url(file_ids["oil"]))
df_stores         = load_csv_from_url(make_drive_url(file_ids["stores"]))
# df_train          = load_csv_from_url(make_drive_url(file_ids["train"])) we dont read it as the file is too big and wont work this way
df_transactions   = load_csv_from_url(make_drive_url(file_ids["transactions"]))


In [2]:
import gdown

# Use our existing function to build the download URL
train_url = make_drive_url(file_ids["train"])

# Download the file using gdown
gdown.download(train_url, "train.csv", quiet=False)

Downloading...
From (original): https://drive.google.com/uc?id=1oEX8NEJPY7wPmSJ0n7lO1JUFYyZjFBRv
From (redirected): https://drive.google.com/uc?id=1oEX8NEJPY7wPmSJ0n7lO1JUFYyZjFBRv&confirm=t&uuid=44205705-d992-4d00-b4de-b74c3a9d2927
To: /content/train.csv
100%|██████████| 5.00G/5.00G [00:56<00:00, 89.0MB/s]


'train.csv'

In [3]:
store_ids = df_stores[df_stores['state'] == 'Guayas']['store_nbr'].unique()
import pandas as pd

chunk_size = 10**6  # 1 million rows at a time
filtered_chunks = []

for chunk in pd.read_csv("train.csv", chunksize=chunk_size):
    chunk_filtered = chunk[chunk['store_nbr'].isin(store_ids)]
    filtered_chunks.append(chunk_filtered)
    del chunk  # Free up memory
df_train = pd.concat(filtered_chunks, ignore_index=True)
df_train = df_train.sample(n=2_000_000).reset_index(drop=True)
del filtered_chunks
# Loads first 5 rows
df_train.head(5)

/tmp/ipython-input-4212995332.py:7: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv("train.csv", chunksize=chunk_size):


,id,date,store_nbr,item_nbr,unit_sales,onpromotion
0,97871318,2016-11-24,29,554049,1.000,False
1,7997314,2013-07-07,35,457928,6.163,NaN
2,45720495,2015-04-29,29,213066,5.000,False
3,40924204,2015-02-10,30,1121515,3.000,False
4,37063236,2014-12-10,30,1333224,3.000,False


In [4]:
df_items.head()

,item_nbr,family,class,perishable
0,96995,GROCERY I,1093,0
1,99197,GROCERY I,1067,0
2,103501,CLEANING,3008,0
3,103520,GROCERY I,1028,0
4,103665,BREAD/BAKERY,2712,1


In [5]:
# 1. Top-3-Familien bestimmen
family_counts = df_items.groupby("family")["item_nbr"].nunique().sort_values(ascending=False)
top3_families = family_counts.head(3).index
print("Top 3 Familien:", top3_families)

# 2. Items dieser Familien herausholen
top3_items = df_items[df_items["family"].isin(top3_families)]["item_nbr"]

# 3. df_train auf diese Items beschränken
df_train_3 = df_train[df_train["item_nbr"].isin(top3_items)]
df_train_3.head()


Top 3 Familien: Index(['GROCERY I', 'BEVERAGES', 'CLEANING'], dtype='object', name='family')


,id,date,store_nbr,item_nbr,unit_sales,onpromotion
0,97871318,2016-11-24,29,554049,1.0,False
2,45720495,2015-04-29,29,213066,5.0,False
3,40924204,2015-02-10,30,1121515,3.0,False
4,37063236,2014-12-10,30,1333224,3.0,False
6,36624208,2014-12-04,30,828204,5.0,False


In [6]:
# 1.  Mount Drive (one-time click to authorize)
from google.colab import drive
drive.mount('/content/drive')

# 2.  Pick a folder inside Drive (create it if it doesn’t exist)
save_path = "/content/drive/MyDrive/Time Series/df_train_3.pkl" # example of path

# 3.  SAVE the cleaned DataFrame
df_train_3.to_pickle(save_path)
print(f"Saved to: {save_path}")

Mounted at /content/drive
Saved to: /content/drive/MyDrive/Time Series/df_train_3.pkl


In [2]:
from google.colab import drive
import pandas as pd
drive.mount('/content/drive')                # re-mount
load_path = "/content/drive/MyDrive/Time Series/df_train_3.pkl" #example of path, use the same as above
df_train = pd.read_pickle(load_path)
print("DataFrame reloaded, ready to go!")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
DataFrame reloaded, ready to go!
